# Import Libraries

In [9]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_auc_score

import re
import string
import nltk
import itertools
import numpy as np

# this allows you to apply a progress bar to pandas operations
from tqdm import tqdm
tqdm.pandas()

# Install Paddle OCR & OPENCV

In [2]:
# Github repo installation of paddle
!python -m pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple
# Install paddle OCR
!pip install paddleocr
# git clone
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
# cv2
!pip install "opencv-python-headless<4.3"


Looking in indexes: https://mirror.baidu.com/pypi/simple
fatal: destination path 'PaddleOCR' already exists and is not an empty directory.


# Import Libraries 2

In [3]:
import cv2
from paddleocr import PaddleOCR, draw_ocr
from matplotlib import pyplot as plt
import os

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': Image.NEAREST,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': Image.BILINEAR,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': Image.BICUBIC,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:39: DeprecationWarning: BOX is deprecated and will be removed in Pillo

# Define ocr_model = PaddleOCR. return text

In [4]:
ocr_model = PaddleOCR(lang='en')

[2022/09/06 09:32:03] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, shape_info_filename=None, precision='fp32', gpu_mem=500, image_dir=None, det_algorithm='DB', det_model_dir='/home/studio-lab-user/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='SVTR_LCNet', rec_model_dir='/home/studio-lab-user/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer', rec_image_shape='3, 48, 320', rec_bat

---    Fused 0 subgraphs into layer_norm op.
---    Fused 0 subgraphs into layer_norm op.


# Read_csv from SWOOP after image & text analysis

In [5]:
# To pull in previusly save data
import pandas as pd
data = pd.read_csv("image_relevant.csv", encoding='latin')
data.head()

,Unnamed: 0,id,description,image,images,relevant,processed_description,text_score
0,0,918707,Boys Among Us Hoodies Super Cool Print Clothes...,968bf5e6c99211c2ad8ac27fa55e4651b941d137,['968bf5e6c99211c2ad8ac27fa55e4651b941d137'],1,boys among us hoodies super cool print clothes...,0.192416
1,1,918708,Cat And Dog Cardboard Printing Jigsaw Puzzle M...,68ece3fab138feb537eee296f3348f3543e23a1d,"['68ece3fab138feb537eee296f3348f3543e23a1d', '...",0,cat dog cardboard printing jigsaw puzzle manuf...,0.030835
2,2,918709,0098 30 Pcs 21pack Sensory 4 Piece Puzzle Fidg...,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,"['a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf', '...",0,0098 30 pcs 21pack sensory 4 piece puzzle fidg...,0.001173
3,3,918710,Boys Sweatshirt Among us 3D Hoodies Cool Fashi...,a278812a832f24a822e2602b91a7b1f5749926cb,['a278812a832f24a822e2602b91a7b1f5749926cb'],1,boys sweatshirt among us 3d hoodies cool fashi...,0.275841
4,4,918711,20cm Among Us Soft Plush Toys Vocal Butt Toy S...,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,['3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a'],0,20cm among us soft plush toys vocal butt toy s...,0.000861


# Slicing 'Image' and 'Relevant' columns and create a dataframe

In [6]:
image_df = data[['image','relevant']]

In [10]:
image_df.head()
len(image_df)

469

# Define words & Scores output from Paadle OCR

In [11]:
def get_words_and_scores_and_add_to_dataframe(row):
    photo = row["image"]
    if photo == "empty":
        return [[],[]]
    final = []
    image_path = f"/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images/{photo}.jpg"
    result = ocr_model.ocr(image_path)
    texts = [res[1][0] for res in result]
    # for word in texts:
    #     final.extend(word.split())
    scores = [res[1][1] *100 for res in result]
    result = [texts, scores]
    return result

# Apply(function, axis=1) to image_df. GPU takes 0.1s, CPU takes 1s

In [12]:
image_df["words"] = image_df.apply(get_words_and_scores_and_add_to_dataframe, axis=1)

[2022/09/06 09:32:50] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/06 09:32:51] ppocr DEBUG: dt_boxes num : 2, elapse : 0.764662504196167
[2022/09/06 09:32:51] ppocr DEBUG: rec_res num  : 2, elapse : 0.14484906196594238
[2022/09/06 09:32:51] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/06 09:32:52] ppocr DEBUG: dt_boxes num : 1, elapse : 0.7796931266784668
[2022/09/06 09:32:52] ppocr DEBUG: rec_res num  : 1, elapse : 0.10230255126953125
[2022/09/06 09:32:52] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/06 09:32:52] ppocr DEBUG: dt_boxes num : 0, elapse : 0.07107996940612793
[2022/09/06 09:32:52] ppocr DEBUG: rec_res num  : 0, elapse : 1.1920928955078125e-06
[2022/09/06 09:32:52] ppocr WARNING: Since the angle classifi

# Result of image_df

In [13]:
image_df.head(50)

,image,relevant,words
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1,"[[AMONGUS], [82.79706239700317]]"
1,68ece3fab138feb537eee296f3348f3543e23a1d,0,"[[zfsz.en.alibaba.com], [92.13441610336304]]"
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0,"[[], []]"
3,a278812a832f24a822e2602b91a7b1f5749926cb,1,"[[Impostor], [97.47896194458008]]"
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0,"[[Press to make a sound], [99.68701601028442]]"
5,9a12a490fcea7313378b1e83425fae86ddc81124,0,"[[], []]"
6,533a514c8c422f93173db706aeb8a2e87603e575,1,"[[AMONG-US, AMONG, AMONC US, YOU, YOU, LOOK, L..."
7,7dc026543aaf6863f0bc810acda21e888cec7693,0,"[[changing, Face], [96.80294394493103, 98.2673..."
8,c24c8919bfeb37dbca1422e373d6f79693efedcd,0,"[[], []]"
9,c74d92fc4f342df4e9a7bf9db1b77ad5cfeea89c,0,"[[], []]"


# Define cleaning_text. Make text lowercase and remove punctuations etc.
## Return is cleaned text and score into the list

In [14]:
import re
import string

def clean_text(row):
    word_list = row["words"][0]
    new_word_list = []
    for word in word_list:
        new_word = word.lower()
        new_word = re.sub("[%s]" % re.escape(string.punctuation), " ", new_word)
        new_word_list.append(new_word)
    return [new_word_list, row["words"][1]]

# Apply apply(function, axis=1)

In [15]:
image_df["words"] = image_df.apply(clean_text, axis=1)

# Result of Clean_text

In [16]:
image_df.head(50)

,image,relevant,words
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1,"[[amongus], [82.79706239700317]]"
1,68ece3fab138feb537eee296f3348f3543e23a1d,0,"[[zfsz en alibaba com], [92.13441610336304]]"
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0,"[[], []]"
3,a278812a832f24a822e2602b91a7b1f5749926cb,1,"[[impostor], [97.47896194458008]]"
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0,"[[press to make a sound], [99.68701601028442]]"
5,9a12a490fcea7313378b1e83425fae86ddc81124,0,"[[], []]"
6,533a514c8c422f93173db706aeb8a2e87603e575,1,"[[among us, among, amonc us, you, you, look, l..."
7,7dc026543aaf6863f0bc810acda21e888cec7693,0,"[[changing, face], [96.80294394493103, 98.2673..."
8,c24c8919bfeb37dbca1422e373d6f79693efedcd,0,"[[], []]"
9,c74d92fc4f342df4e9a7bf9db1b77ad5cfeea89c,0,"[[], []]"


# Targets

In [29]:
## define the target words / phrases
targets = ["among us", "imposter"]

# Get_final_score. each word from text is called sample.
1. calculate the edit_distance between target and sample 
2. calculate the closeness (how similar two words)
3. confidence_score_mean (don't know why confidence_score_mean. Can I use just confidencescore as a score?
4. final score = similarity * confidence score
5. If final score is higher than Max_score =0, max_score is replaced. The highest final score will be the max_score at the end

In [99]:
def get_final_score(row):
    max_score = 0
    text = row["words"][0]
    score = row["words"][1]

    for target in targets:
        for sample in text:
            edit_distance = nltk.edit_distance(target, sample)
            closeness = (len(target)-edit_distance)/len(target)
            confidence_score = score # can't multiply sequence by non-int of type 'float'
            # confidence_score = int(score) # int() argument must be a string, a bytes-like object or a number, not 'list'
            confidence_score = np.mean(score)
            final_score = closeness * confidence_score
            if final_score > max_score:
                max_score = final_score
    return max_score/100

# Calculate OCR_score which is Max_score from get_final_score function

In [100]:
image_df["OCR_score"] = image_df.apply(get_final_score, axis=1)

# display the dataframe sort by OCR_score

In [101]:
image_df = image_df.sort_values(by='OCR_score', ascending=False)
pd.set_option('display.max_rows', None)
len(image_df)

469

# ROCAUC score

In [102]:
RA_score = roc_auc_score(image_df["relevant"], image_df["OCR_score"])

print("Rekognition ROC AUC Score =  %.2f" % (RA_score))

Rekognition ROC AUC Score =  0.97


# Save the dataframe as a csv

In [103]:
image_df.to_csv('/home/studio-lab-user/DS-Projects/OCR/paddle_ocr_v2.csv')

In [104]:
# Dataframe of true positives

tp = image_df.loc[(image_df['relevant'] == 1) & (image_df['OCR_score'] >= 0.5)] 
print(len(tp))

134


In [105]:
# Dataframe of true negatives

tn = image_df.loc[(image_df['relevant'] == 0) & (image_df['OCR_score'] < 0.5)] 
print(len(tn))

317


In [106]:
# Dataframe of false positives

fp = image_df.loc[(image_df['relevant'] == 0) & (image_df['OCR_score'] >= 0.5)] 
print(len(fp))

1


In [107]:
# Dataframe of false negatives

fn = image_df.loc[(image_df['relevant'] == 1) & (image_df['OCR_score'] < 0.5)] 
print(len(fn))

17


In [108]:
# Precision (What % of irdentified positives and true?)
precision = len(tp) / (len(tp) + len(fp))
print(precision)

0.9925925925925926


In [109]:
# PRecall (% of true positives identified)
recall = len(tp) / (len(tp) + len(fn))
print(recall)

0.8874172185430463


In [110]:
fn

,image,relevant,words,OCR_score
395,be4d2abbccf6ce04736e46dc1ca314a540f32f33,1,"[[crewmate, are, 2 impostors, amo], [99.625384...",0.498024
382,34be6e784bf761a746a0b4c159186fab606c7cae,1,"[[imp ostor], [64.4377589225769]]",0.483283
402,0a28a8aed9d8908a9a0801c0056e958961a0c6c5,1,"[[mpostok], [77.09682583808899]]",0.481855
404,b67bc0c0403b35c1487c59fe17cebd61ac1178d6,1,"[[crewmate, are, 2 impostors, amo, e], [99.470...",0.457413
325,a0a10018827f0156be2eca6e2b122eaaabf22a3e,1,"[[imong], [87.93975710868835]]",0.439699
224,9312a0394cdca0f19e08f589fb7d5d10b7b73660,1,"[[g us, s, xvamafil, i m your fabher], [92.502...",0.400763
285,1e2688724847e6ace895252f804fc30258f92b12,1,"[[mong, whoisit, nol711], [56.05505704879761, ...",0.334601
29,d5e94eee21b701f5f3d409bb1b9bc43a4b47096c,1,"[[smang], [61.39713525772095]]",0.230239
375,587864639734fb3fbc29cfd53ff4dfdfba662acd,1,"[[smang], [61.39713525772095]]",0.230239
101,f5dd445f3fff531523efd5fa98a5b1d7700a4aed,1,"[[trick, 2, i d skip vote, that, kini, rgency,...",0.223144


In [111]:
print(image_df.loc[86]['words'])
print(image_df.loc[86]['OCR_score'])

[['wond n2', 'mong us'], [81.32509589195251, 97.85457849502563]]
0.7839110754430294
